ConvLSTM trained on simulated streamflow.

In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt 
from datetime import datetime, timedelta
import netCDF4 as nc
import torch
from torch import nn, utils
from torch.utils.tensorboard import SummaryWriter
from src import load_data, evaluate, conv_lstm, datasets, utils, stgcn
import random
import pickle
import json
import networkx as nx

time_stamp = datetime.now().strftime('%Y%m%d-%H%M%S')
time_stamp

'20190824-162736'

In [2]:
import logging
logger = logging.getLogger()
fhandler = logging.FileHandler(filename='../log.out', mode='a')
chandler = logging.StreamHandler(sys.stdout)
formatter = logging.Formatter('%(asctime)s - {} - %(message)s'.format(time_stamp))
fhandler.setFormatter(formatter)
chandler.setFormatter(formatter)
logger.addHandler(fhandler)
logger.addHandler(chandler)
logger.setLevel(logging.INFO)

In [3]:
USE_CUDA = False
if torch.cuda.is_available():
    print('CUDA Available')
    USE_CUDA = True
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
device = torch.device('cuda:0' if USE_CUDA else 'cpu')
num_devices = torch.cuda.device_count() if USE_CUDA else 0
logger.warning('cuda devices: {}'.format(list(torch.cuda.get_device_name(i) for i in range(num_devices))))
torch.manual_seed(0)
np.random.seed(0)
random.seed(0)

CUDA Available
2019-08-24 16:27:36,921 - 20190824-162736 - cuda devices: ['Tesla V100-SXM2-16GB']


In [4]:
partitioning_strategy = 'distance'  # 'distance' or 'unilabel', see https://arxiv.org/abs/1801.07455
max_hops = 3 if partitioning_strategy == 'distance' else None
rdrs_vars = list(range(8))
agg = ['minmax','minmax','minmax','minmax','sum','minmax','minmax','minmax']
include_month = True
dem, landcover, soil, groundwater = True, True, True, True
landcover_types = None
seq_len = 8
seq_steps = 1

train_start = datetime.strptime('2010-01-01', '%Y-%m-%d') + timedelta(days=seq_len * seq_steps)  # first day for which to make a prediction in train set
train_end = '2012-12-31'
test_start = '2013-01-01'
test_end = '2014-12-31'
spatial_val = False  # Whether the val set does spatial or temporal validation
val_fraction = 0.1 if not spatial_val else None

In [5]:
with open('../data/simulations_shervan/subbasins.geojson', 'r') as f:
     subbasin_shapes = json.loads(f.read())

subbasin_graph = utils.create_subbasin_graph()
subbasin_graph.remove_nodes_from(['sub-1'])
train_subbasins = list(int(n[3:]) for n in subbasin_graph.nodes)
val_subbasins = train_subbasins
test_subbasins = train_subbasins
subbasins = train_subbasins

if partitioning_strategy == 'unilabel':
    train_adjacency = torch.unsqueeze(torch.from_numpy(nx.to_numpy_array(subbasin_graph)), 0).float().to(device)
elif partitioning_strategy == 'distance':  # use distances in upstream-graph, i.e. in reversed downstream-graph
    train_adjacency = utils.create_hop_matrix(subbasin_graph.reverse(), max_hops).float().to(device)
else:
    raise Exception('Unsupported partitioning strategy')
val_adjacency = train_adjacency
test_adjacency = train_adjacency

In [6]:
train_dataset = datasets.SubbasinAggregatedDataset(rdrs_vars, train_subbasins, seq_len, seq_steps, train_start, train_end, aggregate_daily=agg, include_months=include_month, 
                                                   dem=dem, landcover=landcover, soil=soil, groundwater=groundwater, landcover_types=landcover_types)
if spatial_val:
    val_dataset = datasets.SubbasinAggregatedDataset(rdrs_vars, val_subbasins, seq_len, seq_steps, train_start, train_end, aggregate_daily=agg, include_months=include_month, 
                                                     conv_scalers=train_dataset.scalers, dem=dem, landcover=landcover, soil=soil, groundwater=groundwater, landcover_types=landcover_types)
else:
    val_dataset = train_dataset

# Two test datasets: one with spatial and temporal validation (i.e., different graph, different time), and one with only temporal validation (i.e. different time period only)
spatial_test_dataset = None
if test_subbasins != train_subbasins:
    spatial_test_dataset = datasets.SubbasinAggregatedDataset(rdrs_vars, test_subbasins, seq_len, seq_steps, test_start, test_end, aggregate_daily=agg, include_months=include_month, 
                                                              conv_scalers=train_dataset.scalers, dem=dem, landcover=landcover, soil=soil, groundwater=groundwater, landcover_types=landcover_types)
temporal_test_dataset = datasets.SubbasinAggregatedDataset(rdrs_vars, train_subbasins, seq_len, seq_steps, test_start, test_end, aggregate_daily=agg, include_months=include_month, 
                                                           conv_scalers=train_dataset.scalers, dem=dem, landcover=landcover, soil=soil, groundwater=groundwater, landcover_types=landcover_types)

station_subbasins = train_dataset.simulated_streamflow[~pd.isna(train_dataset.simulated_streamflow['StationID'])]['subbasin'].unique()

Using saved dataset in file /home/mgauch/runoff-nn/src/../data/train_test/SubbasinAggregatedDataset_0-1-2-3-4-5-6-7_8-1_2010-01-09000000-2012-12-31_dem__minmax-minmax-minmax-minmax-sum-minmax-minmax-minmax_month_4ee85d401cc849817aa8bc4599cf9756.pkl
Using saved dataset in file /home/mgauch/runoff-nn/src/../data/train_test/SubbasinAggregatedDataset_0-1-2-3-4-5-6-7_8-1_2013-01-01-2014-12-31_dem__minmax-minmax-minmax-minmax-sum-minmax-minmax-minmax_month_4ee85d401cc849817aa8bc4599cf9756.pkl


In [7]:
# Train model
num_epochs = 500
learning_rate = 2e-3
patience = 300
min_improvement = 0.01
best_loss_model = (-1, np.inf, None)
dropout = 0.4
weight_decay = 1e-4

batch_size = 16
temp_kernel_size = 3
model = stgcn.Model(train_dataset.x.shape[2], train_adjacency.shape[0], temp_kernel_size, dropout=dropout).to(device)
if num_devices > 1:
    model = torch.nn.DataParallel(model, device_ids=list(range(num_devices)))
loss_fn = evaluate.NSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

writer = SummaryWriter(comment='STGCN_simulationTraining')
param_description = {'time_stamp': time_stamp, 'batch_size': batch_size, 'loss': loss_fn, 'include_month': include_month, 'aggregate_daily': agg, 'rdrs_vars': rdrs_vars, 'dropout': dropout, 'spatial_validation': spatial_val, 'val_fraction': val_fraction, 'temp_kernel': temp_kernel_size,
                     'optimizer': optimizer, 'lr': learning_rate, 'patience': patience, 'min_improvement': min_improvement, 'x_train_shape': train_dataset.x.shape, 'x_val_shape': val_dataset.x.shape, 'partitioning_strategy': partitioning_strategy, 'max_hops': max_hops,
                     'spatial_x_test_shape': spatial_test_dataset.x.shape if spatial_test_dataset is not None else '', 'temporal_x_test_shape': temporal_test_dataset.x.shape, 'num_epochs': num_epochs, 'seq_len': seq_len, 'seq_steps': seq_steps, 'train_start': train_start, 'train_end': train_end, 'weight_decay': weight_decay, 
                     'landcover_types': landcover_types, 'test_start': test_start, 'test_end': test_end, 'model': str(model).replace('\n','').replace(' ', ''),}
writer.add_text('Parameter Description', str(param_description))
str(param_description)

"{'time_stamp': '20190824-162736', 'batch_size': 16, 'loss': NSELoss(), 'include_month': True, 'aggregate_daily': ['minmax', 'minmax', 'minmax', 'minmax', 'sum', 'minmax', 'minmax', 'minmax'], 'rdrs_vars': [0, 1, 2, 3, 4, 5, 6, 7], 'dropout': 0.4, 'spatial_validation': False, 'val_fraction': 0.1, 'temp_kernel': 3, 'optimizer': Adam (\nParameter Group 0\n    amsgrad: False\n    betas: (0.9, 0.999)\n    eps: 1e-08\n    lr: 0.002\n    weight_decay: 0.0001\n), 'lr': 0.002, 'patience': 300, 'min_improvement': 0.01, 'x_train_shape': torch.Size([1088, 8, 178, 724]), 'x_val_shape': torch.Size([1088, 8, 178, 724]), 'partitioning_strategy': 'distance', 'max_hops': 3, 'spatial_x_test_shape': '', 'temporal_x_test_shape': torch.Size([730, 8, 178, 724]), 'num_epochs': 500, 'seq_len': 8, 'seq_steps': 1, 'train_start': datetime.datetime(2010, 1, 9, 0, 0), 'train_end': '2012-12-31', 'weight_decay': 0.0001, 'landcover_types': None, 'test_start': '2013-01-01', 'test_end': '2014-12-31', 'model': 'Model((d

In [8]:
if spatial_val:
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size, shuffle=True, pin_memory=True, drop_last=False)
    val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size, shuffle=True, pin_memory=True, drop_last=False)
else:
    val_indices = np.random.choice(len(train_dataset), size=int(val_fraction * len(train_dataset)), replace=False)
    train_indices = list(i for i in range(len(train_dataset)) if i not in val_indices)
    train_sampler = torch.utils.data.SubsetRandomSampler(train_indices)
    val_sampler = torch.utils.data.SubsetRandomSampler(val_indices)
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size, sampler=train_sampler, pin_memory=True, drop_last=False)
    val_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size, sampler=val_sampler, pin_memory=True, drop_last=False)
    
if spatial_test_dataset is not None:
    spatial_test_dataloader = torch.utils.data.DataLoader(spatial_test_dataset, batch_size, shuffle=False, pin_memory=True, drop_last=False)
temporal_test_dataloader = torch.utils.data.DataLoader(temporal_test_dataset, batch_size, shuffle=False, pin_memory=True, drop_last=False)

In [9]:
# Subbasins with constant streamflow will divide by zero in loss calculation. Calculate loss without them.
train_non_constant_subbasin_mask = ((train_dataset.y_sim.min(dim=0)[0] - train_dataset.y_sim.max(dim=0)[0]) != 0).to(device)
val_non_constant_subbasin_mask = ((val_dataset.y_sim.min(dim=0)[0] - val_dataset.y_sim.max(dim=0)[0]) != 0).to(device)

y_train_means = train_dataset.y_sim_means[train_non_constant_subbasin_mask].to(device)
y_val_means = val_dataset.y_sim_means[val_non_constant_subbasin_mask].to(device)

In [10]:
torch.manual_seed(0)
np.random.seed(0)
for epoch in range(num_epochs):
    model.train()

    train_losses = torch.tensor(0.0)
    for i, train_batch in enumerate(train_dataloader):
        y_pred = model(train_batch['x'].permute(0,2,1,3).to(device), train_adjacency)
        train_loss = loss_fn(y_pred[:,train_non_constant_subbasin_mask], train_batch['y_sim'][:,train_non_constant_subbasin_mask].to(device), means=y_train_means)
        
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
        
        train_losses += train_loss.detach()
        
    train_loss = (train_losses / len(train_dataloader)).item()
    print('Epoch', epoch, 'mean train loss:\t{}'.format(train_loss))
    writer.add_scalar('loss_nse', train_loss, epoch)
    
    model.eval()
    val_losses = torch.tensor(0.0)
    for i, val_batch in enumerate(val_dataloader):
        y_pred = model(val_batch['x'].permute(0,2,1,3).to(device), val_adjacency).detach()
        val_losses += loss_fn(y_pred[:,val_non_constant_subbasin_mask], val_batch['y_sim'][:,val_non_constant_subbasin_mask].to(device), means=y_val_means).detach()
            
    val_loss = (val_losses / len(val_dataloader)).item()
    print('Epoch', epoch, 'mean val loss:\t{}'.format(val_loss))
    writer.add_scalar('loss_nse_val', val_loss, epoch)
    
    if val_loss < best_loss_model[1] - min_improvement:
        best_loss_model = (epoch, val_loss, model.state_dict())  # new best model
        load_data.pickle_model('STGCN_simulationTraining', model, 'allStations', time_stamp, model_type='torch.dill')
    elif epoch > best_loss_model[0] + patience:
        print('Patience exhausted in epoch {}. Best val-loss was {}'.format(epoch, best_loss_model[1]))
        break
    
print('Using best model from epoch', str(best_loss_model[0]), 'which had loss', str(best_loss_model[1]))
model.load_state_dict(best_loss_model[2])
load_data.save_model_with_state('STGCN_simulationTraining', best_loss_model[0], model, optimizer, time_stamp, use_dill=True)

Epoch 0 mean train loss:	8.42356014251709
Epoch 0 mean val loss:	1.5612108707427979
Saved model as /home/mgauch/runoff-nn/src/../pickle/models/STGCN_simulationTraining_allStations_20190824-162736.pkl
Epoch 1 mean train loss:	1.6880286931991577
Epoch 1 mean val loss:	1.534422755241394
Saved model as /home/mgauch/runoff-nn/src/../pickle/models/STGCN_simulationTraining_allStations_20190824-162736.pkl
Epoch 2 mean train loss:	1.6651135683059692
Epoch 2 mean val loss:	1.5775805711746216
Epoch 3 mean train loss:	1.5572295188903809
Epoch 3 mean val loss:	1.4942139387130737
Saved model as /home/mgauch/runoff-nn/src/../pickle/models/STGCN_simulationTraining_allStations_20190824-162736.pkl
Epoch 4 mean train loss:	1.5269315242767334
Epoch 4 mean val loss:	1.4781752824783325
Saved model as /home/mgauch/runoff-nn/src/../pickle/models/STGCN_simulationTraining_allStations_20190824-162736.pkl
Epoch 5 mean train loss:	1.489630937576294
Epoch 5 mean val loss:	1.5386521816253662
Epoch 6 mean train loss:

In [11]:
del y_train_means, y_val_means, y_pred, train_non_constant_subbasin_mask, val_non_constant_subbasin_mask
if USE_CUDA:
    torch.cuda.empty_cache()

In [12]:
logger.warning('predicting')
model.eval()

temporal_test_predictions = []  # test on train graph but different time
for i, test_batch in enumerate(temporal_test_dataloader):
    pred = model(test_batch['x'].permute(0,2,1,3).to(device), train_adjacency).detach().cpu()
    temporal_test_predictions.append(pred)
predictions = torch.cat(temporal_test_predictions)

if spatial_test_dataset is not None:
    spatial_test_predictions = []  # test on different graph, different time
    for i, test_batch in enumerate(spatial_test_dataloader):
        pred = model(test_batch['x'].permute(0,2,1,3).to(device), test_adjacency).detach().cpu()
        spatial_test_predictions.append(pred)
    predictions = torch.cat([torch.cat(spatial_test_predictions), predictions], dim=1)

2019-08-24 18:17:31,914 - 20190824-162736 - predicting


In [13]:
actuals = temporal_test_dataset.data_streamflow.copy()
if len(actuals['date'].unique()) != len(predictions):
    print('Warning: length of prediction {} and actuals {} does not match.'.format(len(predictions), len(actuals['date'].unique())))

nse_dict, nse_sim_dict = {}, {}
mse_dict, mse_sim_dict = {}, {}
predictions_df = pd.DataFrame(columns=actuals.columns)
predictions_df['is_test_subbasin'] = False
all_subbasins = test_subbasins + train_subbasins if train_subbasins != test_subbasins else train_subbasins
for i in range(len(all_subbasins)):
    subbasin = all_subbasins[i]
    station = None
    subbasin_sim = temporal_test_dataset.simulated_streamflow[temporal_test_dataset.simulated_streamflow['subbasin'] == subbasin].set_index('date')
    if subbasin in station_subbasins:
        station = subbasin_sim['StationID'].values[0]
        act = actuals[actuals['station'] == station].set_index('date')['runoff']
    if predictions.shape[0] != subbasin_sim.shape[0]:
        print('Warning: length of prediction {} and actuals {} does not match for subbasin {}. Ignoring excess actuals.'.format(len(predictions), len(subbasin_sim), subbasin))
        subbasin_sim = subbasin_sim.iloc[:predictions.shape[0]]
        if station is not None:
            act = act.iloc[:predictions.shape[0]]
            
    pred = pd.DataFrame({'runoff': predictions[:,i]}, index=subbasin_sim.index)
    pred['subbasin'] = subbasin
    pred['station'] = station
    pred['is_test_subbasin'] = subbasin in test_subbasins
    predictions_df = predictions_df.append(pred.reset_index(), sort=True)
    subbasin_type = 'test' if subbasin in test_subbasins else 'train'
    nse_sim, mse_sim = evaluate.evaluate_daily('Sub{}'.format(subbasin), pred['runoff'], subbasin_sim['simulated_streamflow'], writer=writer, group=subbasin_type)
    nse_sim_dict[subbasin] = nse_sim
    mse_sim_dict[subbasin] = mse_sim

    if station is not None:
        nse, mse = evaluate.evaluate_daily(station, pred['runoff'], act, writer=writer)
        nse_dict[subbasin] = nse
        mse_dict[subbasin] = mse
        print(station, subbasin, '\tNSE:', nse, '\tMSE:', mse, '(clipped to 0)')
    print(subbasin, '\tNSE sim:', nse_sim, '\tMSE sim:', mse_sim)

/home/mgauch/miniconda3/envs/gwf/lib/python3.7/site-packages/pandas/plotting/_converter.py:129: FutureWarning: Using an implicitly registered datetime converter for a matplotlib plotting method. The converter was registered by pandas on import. Future versions of pandas will require you to explicitly register matplotlib converters.

To register the converters:
	>>> from pandas.plotting import register_matplotlib_converters
	>>> register_matplotlib_converters()
  warnings.warn(msg, FutureWarning)


1 	NSE sim: -1.1727618742662211 	MSE sim: 2513728.901788604
10 	NSE sim: 0.42308960776813276 	MSE sim: 3.2882941886140893
100 	NSE sim: -0.2668894017611101 	MSE sim: 37.4141249124941
101 	NSE sim: 0.20204267896471206 	MSE sim: 2.052709154700926
102 	NSE sim: 0.09034372126025636 	MSE sim: 441.2818382305844
103 	NSE sim: 0.3276464405333678 	MSE sim: 3.6702063077117164
104 	NSE sim: 0.32329681479874395 	MSE sim: 41.341461435000234
105 	NSE sim: -0.1783538607613877 	MSE sim: 8.220395963983146
106 	NSE sim: 0.3054467186822679 	MSE sim: 31.314101133609228
107 	NSE sim: 0.3587962925326471 	MSE sim: 1.6316731868655994
108 	NSE sim: 0.1885067232032881 	MSE sim: 6.914247430157964
109 	NSE sim: 0.16661271210695183 	MSE sim: 258.91980177462966
11 	NSE sim: 0.40555615045709004 	MSE sim: 7.254082781712348
110 	NSE sim: -1.1256454625275563 	MSE sim: 62.14327534238617
111 	NSE sim: -1.1068691650626126 	MSE sim: 10.005935071007757
112 	NSE sim: -0.8872219680534716 	MSE sim: 5.274340099580292
113 	NSE s

/home/mgauch/miniconda3/envs/gwf/lib/python3.7/site-packages/hydroeval/objective_functions.py:31: RuntimeWarning: divide by zero encountered in true_divide
  np.sum((evaluation - np.mean(evaluation)) ** 2, dtype=np.float64))


5 	NSE sim: 0.22184365194610822 	MSE sim: 18.873757177484805
50 	NSE sim: 0.05582245665710239 	MSE sim: 2061.867963806055
500 	NSE sim: -0.919292015201721 	MSE sim: 22.517713846326554
501 	NSE sim: 0.07146989279621674 	MSE sim: 0.03698188603032226
502 	NSE sim: 0.07577585319773184 	MSE sim: 0.09635272172936775
503 	NSE sim: 0.23972632818506012 	MSE sim: 0.02051753092473459
504 	NSE sim: 0.21138979044680206 	MSE sim: 0.005307472219991988
505 	NSE sim: 0.18522250298036635 	MSE sim: 2.31695549125428
506 	NSE sim: 0.28974491161762594 	MSE sim: 24.975536327109108
507 	NSE sim: 0.2500580039041207 	MSE sim: 0.010445721121614036
508 	NSE sim: 0.31061794875082793 	MSE sim: 1.7965516908309744
509 	NSE sim: 0.24772630477130408 	MSE sim: 43.2885456715008
51 	NSE sim: -1.0284405712484919 	MSE sim: 46.782166621411974
510 	NSE sim: 0.37714396342827516 	MSE sim: 0.6024545027469854
511 	NSE sim: 0.3420595982764342 	MSE sim: 0.47924466247082004
512 	NSE sim: 0.2680269260392786 	MSE sim: 171.572278053740

/home/mgauch/miniconda3/envs/gwf/lib/python3.7/site-packages/hydroeval/objective_functions.py:31: RuntimeWarning: divide by zero encountered in true_divide
  np.sum((evaluation - np.mean(evaluation)) ** 2, dtype=np.float64))


526 	NSE sim: -4.870933801602193 	MSE sim: 0.0022807017594055432
527 	NSE sim: 0.4094725312030364 	MSE sim: 0.009682665539201907
528 	NSE sim: 0.40696595387323653 	MSE sim: 0.04005508437589127
529 	NSE sim: 0.4923098842955418 	MSE sim: 0.005233100077279163
53 	NSE sim: 0.44803147419105405 	MSE sim: 40.041294722356156
530 	NSE sim: 0.3480174024633228 	MSE sim: 3.440925785956306
531 	NSE sim: -0.501808677285916 	MSE sim: 1.795044636794275
532 	NSE sim: -0.40786851803704827 	MSE sim: 0.4627485995650656
533 	NSE sim: -2.1756836581223626 	MSE sim: 0.006238681725632859
534 	NSE sim: 0.44424035739260603 	MSE sim: 0.33977576585026625
535 	NSE sim: 0.3359557726831639 	MSE sim: 0.20639387386766325
536 	NSE sim: 0.3385903787612282 	MSE sim: 1.7718963615797736
537 	NSE sim: 0.2862863030002941 	MSE sim: 1.1526295927961954
538 	NSE sim: -0.6532484604323587 	MSE sim: 283.0101422585241
539 	NSE sim: 0.2167475393761713 	MSE sim: 0.015222510142272851
54 	NSE sim: 0.49890567530850705 	MSE sim: 2.66558844

In [14]:
def print_nse_mse(name, nse_dict, mse_dict, subbasins):
    nses = list(nse_dict[s] for s in subbasins)
    mses = list(mse_dict[s] for s in subbasins)
    print(name, 'Median NSE (clipped to 0)', np.nanmedian(nses), '/ Min', np.nanmin(nses), '/ Max', np.nanmax(nses))
    print(' ' * len(name), 'Median MSE (clipped to 0)', np.nanmedian(mses), '/ Min', np.nanmin(mses), '/ Max', np.nanmax(mses))
    
    return np.nanmedian(nses)

nse_median_sim_temporal = print_nse_mse('Temporal test sim', nse_sim_dict, mse_sim_dict, train_subbasins)
nse_median_stations_temporal = print_nse_mse('Stations temporal test', nse_dict, mse_dict, list(s for s in station_subbasins if s in train_subbasins))
if spatial_test_dataset is not None:
    nse_median_sim_spatial = print_nse_mse('Spatial test sim', nse_sim_dict, mse_sim_dict, test_subbasins)
    nse_median_stations_spatial = print_nse_mse('Stations spatial test', nse_dict, mse_dict, list(s for s in station_subbasins if s in test_subbasins))
    writer.add_scalar('nse_median_sim', nse_median_sim_spatial)
    writer.add_scalar('nse_median_stations_spatial', nse_median_stations_spatial)
writer.add_scalar('nse_median_sim_temporal', nse_median_sim_temporal)
writer.add_scalar('nse_median_stations_temporal', nse_median_stations_temporal)

Temporal test sim Median NSE (clipped to 0) 0.2900719843862446 / Min -4.870933801602193 / Max 0.608878143335952
                  Median MSE (clipped to 0) 9.866500425005162 / Min 0.0022807017594055432 / Max 2527317.470117438
Stations temporal test Median NSE (clipped to 0) 0.1174879790393939 / Min -1.5492974214425796 / Max 0.462713177603586
                       Median MSE (clipped to 0) 153.31606860400962 / Min 14.522882848690957 / Max 52080.44405645281


In [15]:
nse_dict

{676: -0.0164852584597881,
 677: -0.437725688115308,
 678: 0.3033590483573616,
 680: 0.462713177603586,
 681: 0.41701216663040586,
 682: 0.16060804440322785,
 683: 0.14415120036165263,
 684: -0.0007919487160139571,
 685: 0.008788500901953156,
 686: 0.2205354045914616,
 687: -0.1610827176905223,
 688: 0.024028964684809972,
 689: -0.9804000418766257,
 690: -0.31299244854885355,
 691: -0.2241903677492456,
 692: -0.8553572906548577,
 693: 0.11488348409215599,
 694: 0.18698682898916963,
 695: 0.2316591490912303,
 696: 0.28418289177875056,
 697: 0.22233873257008385,
 698: 0.20307590007019038,
 699: -0.1792368122614656,
 700: -0.21756723826073499,
 701: 0.26208454999433994,
 702: -0.7460064220105433,
 703: -0.03295025368097826,
 704: 0.19431104399986932,
 705: 0.12009247398663181,
 706: 0.12635093162055633,
 707: -0.26028611589539574,
 709: 0.16209209533321656,
 710: 0.15860241921636975,
 712: 0.12387573725302459,
 713: 0.2644082923552956,
 714: 0.20207770286524818,
 715: -0.2471627925394555,

In [16]:
writer.close()

In [17]:
save_df = pd.merge(predictions_df.rename({'runoff': 'prediction'}, axis=1), 
                   temporal_test_dataset.simulated_streamflow, on=['date', 'subbasin'])
save_df = pd.merge(save_df, actuals.rename({'runoff': 'actual'}, axis=1), how='left', on=['date', 'station'])\
            [['date', 'subbasin', 'station', 'prediction', 'actual', 'simulated_streamflow', 'is_test_subbasin']]
load_data.pickle_results('STGCN_simulationTraining', save_df, time_stamp)

'STGCN_simulationTraining_20190824-162736.pkl'

In [18]:
_ = print(train_subbasins), print(val_subbasins), print(test_subbasins)

[1, 10, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 11, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 12, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 13, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 14, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 15, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 16, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 17, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 19, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 2, 20, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 21, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 22, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 23, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 24, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 25, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 26, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 27, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 28, 280, 281, 282, 28

In [19]:
list(k for k in list(nse_dict.keys()) if k in test_subbasins)

[676,
 677,
 678,
 680,
 681,
 682,
 683,
 684,
 685,
 686,
 687,
 688,
 689,
 690,
 691,
 692,
 693,
 694,
 695,
 696,
 697,
 698,
 699,
 700,
 701,
 702,
 703,
 704,
 705,
 706,
 707,
 709,
 710,
 712,
 713,
 714,
 715,
 716,
 717,
 718,
 719,
 720,
 721,
 722,
 723,
 724]

In [20]:
datetime.now().strftime('%Y%m%d-%H%M%S')

'20190824-181926'